In [1]:
with open("./data/processed/Agentic_Design_Patterns.mmd", "r") as f:
    full_text = f.read()

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

rec_splitter = RecursiveCharacterTextSplitter(chunk_size=1750, chunk_overlap=250)
rec_chunks = rec_splitter.split_text(full_text)

In [3]:
len(rec_chunks)

614

In [80]:
chunk = rec_chunks[50]
chunk

"## References  \n\n\nReferences1. LangChain Documentation on LCEL: https://python.langchain.com/v0.2/docs/core_modules/expression_language/2. LangGraph Documentation: https://langchain- ai.github.io/langgraph/3. Prompt Engineering Guide - Chaining Prompts: https://www.promptingguide.ai/techniques/chaining4. OpenAI API Documentation (General Prompting Concepts): https://platform.openai.com/docs/guides/gpt/prompting5. Crew AI Documentation (Tasks and Processes): https://docs.crewai.com/6. Google AI for Developers (Prompting Guides): https://cloud.google.com/discover/what- is- prompt- engineering?hl=en7. Vertex Prompt Optimizer https://cloud.google.com/vertex- ai/generative- ai/docs/learn/prompts/prompt- optimizer\n\n\n\n\n## Chapter 2: Routing  \n\n\n## Routing Pattern Overview  \n\n\nWhile sequential processing via prompt chaining is a foundational technique for executing deterministic, linear workflows with language models, its applicability is limited in scenarios requiring adaptive 

In [81]:
len(chunk) / 4

425.0

In [59]:
from openai import OpenAI
from openai import RateLimitError, APIError, Timeout
from pydantic import BaseModel, Field
from dotenv import load_dotenv
from time import sleep

load_dotenv()

client = OpenAI()

book_name = "Agentic Design Patterns"

# -----------------------------------------------------------------------------
# 1. Pydantic Model for Structured Output
# -----------------------------------------------------------------------------
class ChunkMetadata(BaseModel):
    semantic_title: str = Field(description="A human-readable short title for the chunk")
    context_expansion: str = Field(description="Using the cached full book, generate a short context expansion for this chunk, no more than 2 sentences, that explains the broader context and patterns from the book relevant to this text.")
    section_header: str = Field(description="The section title or chapter the chunk belongs to")
    keywords: list[str] = Field(description="Important keywords describing this chunk")

MAX_RETRIES = 5
BASE_SLEEP = 18  # seconds

data_chunks = []

for chunk in rec_chunks[:5]:
    
    user_message = {
        "role": "user",
        "content": f"Enrich the following chunk using the full book text:\n\n{chunk}"
    }

    system_prompt={
        "role": "system",
        "content": f"""
        You are a helpful RAG assistant. This is the full book '{book_name}'.

        {full_text}
    """
    }
    
    retries = 0
    
    while retries < MAX_RETRIES:
        try:    
            response = client.responses.parse(
                model="gpt-5-nano",
                input=[
                    system_prompt,  # cached full book
                    user_message           # new chunk
                ],
                text_format=ChunkMetadata,
            )

            parsed = response.output_parsed
            # print(parsed)
            print(response.usage.input_tokens_details.cached_tokens)
            chunk_metada = {}
            chunk_metada["context_expansion"] = parsed.context_expansion
            chunk_metada["semantic_title"] = parsed.semantic_title
            chunk_metada["section_header"] = parsed.section_header
            chunk_metada["keywords"] = parsed.keywords
            
            data_chunks.append({
                "chunk": chunk,
                "chunk_metada": chunk_metada
            })
            
            sleep(BASE_SLEEP)
            break
        except (RateLimitError, APIError, TimeoutError) as e:
            retries += 1
            sleep_time = BASE_SLEEP * retries
            print(f"Error: {e}")
            sleep(sleep_time)

169472
0
169472
Error: <html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
<hr><center>cloudflare</center>
</body>
</html>
0
169600


In [ ]:
chunk

'by design. And above all, we must Inspire Trust, by being transparent about our methods and accountable for our outcomes.'

In [ ]:
parsed.context_expansion

"The passage underscores the book's focus on governance in AI design. It foreshadows a commitment to transparency and accountability across agentic systems and patterns."

In [ ]:
parsed.semantic_title

'Trust and Transparency as Design Tenets'

In [ ]:
parsed.section_header

"A Thought Leader's Perspective: Power and Responsibility"

In [ ]:
parsed.keywords

['trust',
 'transparency',
 'accountability',
 'guardrails',
 'HITL',
 'ethics',
 'design patterns',
 'agentic systems']

In [ ]:
response.usage.input_tokens_details.cached_tokens

0

In [60]:
import json
import os

# Ensure the directory exists
output_dir = "./data/processed/"
os.makedirs(output_dir, exist_ok=True)

output_path = os.path.join(output_dir, "enriched_chunks.json")

# Save the data_chunks list as a JSON file
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(data_chunks, f, ensure_ascii=False, indent=4)

print(f"Saved {len(data_chunks)} enriched chunks to {output_path}")


Saved 5 enriched chunks to ./data/processed/enriched_chunks.json


In [4]:
import json
import os

# Ensure the directory exists
enriched_chunks_path = "./data/processed/enriched_chunks.json"

with open(enriched_chunks_path, "r", encoding="utf-8") as f:
    enriched_chunks = json.load(f)
    
len(enriched_chunks)

614